In [1]:
import numpy as np
import tensorflow as tf
import phe   # PySyft Homomorphic Encryption Library
from syft.frameworks.torch.mpc import spdz

# Simulated client data
client_data = {
    "client1": np.random.rand(100, 784),
    "client2": np.random.rand(100, 784),
    # ... more clients
}

# Model architecture
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(784,)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

# Encryption setup
public_key, private_key = phe.generate_paillier_keypair()
encryptor = public_key.encryptor()
decryptor = private_key.decryptor()

# Federated Learning loop
for epoch in range(num_epochs):
    model_weights = []  # Store encrypted model weights from clients

    for client, data in client_data.items():
        # Encrypt data locally
        encrypted_data = [encryptor.encrypt(x) for x in data]

        # Perform local training
        local_model = train_local_model(encrypted_data, model)
        encrypted_weights = encryptor.encrypt(local_model.get_weights())

        # Store encrypted model weights
        model_weights.append(encrypted_weights)

    # Aggregate encrypted weights
    aggregated_weights = spdz.sum_tensors(*model_weights)

    # Decrypt and update global model
    decrypted_weights = decryptor.decrypt(aggregated_weights)
    global_model.set_weights(decrypted_weights)

# Perform inference with encrypted input image
input_image = np.random.rand(1, 784)
encrypted_input = encryptor.encrypt(input_image.flatten())
output = global_model.predict(encrypted_input)  # Decryption happens in the client


ModuleNotFoundError: No module named 'phe'